In [5]:
import numpy as np
import prepare_ml_dataset
import train_nn_model

In [6]:
energy = '51767680'
species = 'h'

In [7]:
directories, dataset_csv, data_settings = prepare_ml_dataset.initializ_var(energy, species)


In [14]:
df_full, index_good, data_settings["feature_history_names"] = prepare_ml_dataset.create_df_full('rel05', data_settings["y_name"] , data_settings["y_name_log"] ,  data_settings["feature_names"] , data_settings["dL01"] , data_settings["number_history"] , data_settings["history_resolution"] , data_settings["average_time"], data_settings["forecast"] , directories["data_dir"] )


Reading csv data for probeb

Dask Apply:   0%|          | 0/41 [00:00<?, ?it/s]

26-Feb-25 18:06:31: /home/jliao/anaconda3/lib/python3.11/site-packages/dask/utils.py:1226: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  return getattr(__obj, self.method)(*args, **kwargs)

26-Feb-25 18:06:31: /home/jliao/anaconda3/lib/python3.11/site-packages/dask/utils.py:1226: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  return getattr(__obj, self.method)(*args, **kwargs)

26-Feb-25 18:06:31: /home/jliao/anaconda3/lib/python3.11/site-packages/dask/utils.py:1226: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  return getattr(__obj, self.method)(*args, **kwargs)

26-Feb-25 18:06:31: /home/jl

7.868543463017787 1.040252310912895 4.4543978869653404 6.8282911521048915
19.92544708333316 -19.85855629142716 0.033445395953000556 39.78400337476032
78.0 -232.8000030517578 -77.4000015258789 310.8000030517578
348.0 0.0 174.0 348.0
379.0 0.0 189.5 379.0
2471.39990234375 3.400000095367432 1237.3999512195587 2467.9999022483826
255.0 64.5999984741211 159.79999923706055 190.4000015258789
8.3 0.0 4.15 8.3
56.76599884033203 0.050000000745058 28.407999420538545 56.71599883958697
72.12999725341797 0.1359999924898148 36.13299862295389 71.99399726092815
927.0999755859376 243.6999969482422 585.3999862670898 683.3999786376954
32.51399993896484 -27.52799987792969 2.493000030517573 60.04199981689453
31.9219970703125 -38.81400299072266 -3.4460029602050817 70.73600006103516


26-Feb-25 18:06:35: /home/jliao/workspace/GitHub/ml_ringcurrent_ion/prepare_ml_dataset.py:144: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  for k in range(m_history):

26-Feb-25 18:06:35: /home/jliao/workspace/GitHub/ml_ringcurrent_ion/prepare_ml_dataset.py:144: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  for k in range(m_history):

26-Feb-25 18:06:35: /home/jliao/workspace/GitHub/ml_ringcurrent_ion/prepare_ml_dataset.py:144: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.ins

In [33]:
n_history_total_days = 7
average_time = 300
n_history_total = n_history_total_days*24*60*60/average_time
m_history = int(n_history_total/24 + 1)
history_resolution = 2*3600
m_history = int(n_history_total/(history_resolution/average_time) + 1)
feature_names=[ 'scaled_symh', 'scaled_ae','scaled_asyh', 'scaled_asyd']
m_feature = len(feature_names)

index_difference = history_resolution/average_time
feature_history_names = ["" for x in range(m_feature*m_history)]
ihf = 0
index0 = int(n_history_total)
index1 = df_full.index[-1]

In [34]:
print(index0,index1, m_feature)


2016 1313855 4


In [36]:
df_history = np.empty((index1-index0+1, m_feature))


array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       ...,
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])

In [9]:
x_train, x_valid, x_test, y_train, y_valid, y_test  = prepare_ml_dataset.load_csv_data(dataset_csv)


In [10]:
print(x_train.shape, x_valid.shape, x_test.shape, y_train.shape, y_valid.shape, y_test.shape)

(475943, 346) (119522, 346) (116124, 346) (475943, 3) (119522, 3) (116124, 3)


In [11]:
y_train

,Unnamed: 0.1,Unnamed: 0,0
0,0,0,2.562229
1,1,1,2.628978
2,2,2,3.058719
3,3,3,3.291276
4,4,4,3.487199
...,...,...,...
475938,475938,475938,5.025992
475939,475939,475939,5.047016
475940,475940,475940,5.100988
475941,475941,475941,5.146533


In [ ]:
x_train, x_valid, x_test, y_train, y_valid, y_test = prepare_ml_dataset.prepare_ml_dataset(energy, species)

print(x_train.shape, x_valid.shape, x_test.shape, y_train.shape, y_valid.shape, y_test.shape)

In [ ]:
para_name = "learning_rate"
para_set = [1.e-4, 1.5e-3, 1.e-3]

final_train_loss = np.zeros(len(para_set))
final_valid_loss = np.zeros(len(para_set))
total_history = dict()
valid_r2s = np.zeros(len(para_set))

for ipara in range(len(para_set)):
    parameter = para_set[ipara]

    model, history, valid_r2 = train_nn_model.nn_model(x_train, y_train, x_valid, y_valid, data_settings["y_name_log"], 
                                                       output_dir = directories["training_output_dir"] , n_neurons = 18, patience = 32,
                                                       learning_rate = parameter, epochs = 10, batch_size = 8)
    
    total_history[str(parameter)] = history.history
    final_train_loss[ipara] = history.history['loss'][-1]
    final_valid_loss[ipara] = history.history['val_loss'][-1]
    valid_r2s[ipara] = valid_r2